In [1]:
from langchain_core.prompts import load_prompt

from lib import *
from syslib import has_db, get_db, save_db, get_faiss_db, save_faiss_docs
from prompts import *
import re

In [2]:
llm = get_llm()
emb = get_emb()

In [3]:
mTests = {}
mEntities = {}
mFeature = {}
gen_steps = []
tests = []
entities = []
features = []


Список тестов

In [4]:
print('retrieving tests')
tests_docs = get_zephyr('./docs/zephyr.csv')
mTests = {}
for test in tests_docs:
    key = test.page_content.split(";")[3].replace("Expected Result: ", "")
    mTests[key] = test.page_content
    test.metadata['test_key'] = key

mTests = save_db("./pickled/mTests.pkl", mTests)

retrieving tests
loaded 440 tests.


In [6]:
if has_db("./pickled/tests.pkl"):
    tests = get_faiss_db("./pickled/tests.pkl", emb)
else:
    tests = save_faiss_docs("./pickled/tests.pkl", tests_docs, emb)

Список сгенерированных шагов

In [7]:
if has_db("./pickled/steps.pkl"):
    gen_steps = get_faiss_db("./pickled/steps.pkl", emb)
else:
    print('retrieving steps')
    gen_steps_docs = [
        *get_splitted_doc(file_path='../monoscript/gen-steps.txt', separator="\n\n"),
        *get_doc(file_path='../monoscript/manual-steps.txt'),
    ]
    gen_steps = save_faiss_docs("./pickled/steps.pkl", gen_steps_docs, emb)

./pickled/steps.pkl exists, reusing...


Список сушностей, dto

In [8]:
print('retrieving entities')

entities_docs = [
    *get_docs(file_path='../monoscript/src/main/java/ru/sber/cb/ekp/avtokflekp/subsystem', glob='**/entity/**/*.java'),
    *get_docs(file_path='../monoscript/src/main/java/ru/sber/cb/ekp/avtokflekp/subsystem', glob='**/dto/**/*.java')
]

print(f"len entities_docs: {len(entities_docs)}")

pat2 = r'.+?(\w+).java$'

for doc in entities_docs:
    matcher2 = re.match(pat2, doc.metadata['source'])
    if matcher2:
        doc.metadata['class_key'] = matcher2.group(1)
        mEntities[matcher2.group(1)] = doc
    else:
        print('entity doc error')

retrieving entities
len entities_docs: 66


In [9]:
if has_db("./pickled/entities.pkl"):
    entities = get_faiss_db("./pickled/entities.pkl", emb)
else:
    entities = save_faiss_docs("./pickled/entities.pkl", entities_docs, emb)

./pickled/entities.pkl exists, reusing...


Список features

In [10]:
print('retrieving features')
pat2 = r'.+?(\w+).feature$'
features_docs = get_docs(file_path='../monoscript/src/test/resources/features', glob='**/*.feature')
print(f"len features_docs: {len(features_docs)}")

for feat in features_docs:
    matcher = re.match(pat2, feat.metadata['source'])
    if matcher:
        feat.metadata['test_key'] = matcher.group(1)
        mFeature["AVTOKFLEKP-" + matcher.group(1)] = feat.page_content
    else:
        print(feat)
        print('feature doc error')

retrieving features
len features_docs: 42


In [11]:
if has_db("./pickled/features.pkl"):
    features = get_faiss_db("./pickled/features.pkl", emb)
else:
    features = save_faiss_docs("./pickled/features.pkl", features_docs, emb)

./pickled/features.pkl exists, reusing...


In [12]:
retr_tests = tests.as_retriever(
    search_kwargs={'k': 5},
    search_type="similarity"
)

retr_steps = gen_steps.as_retriever(
    search_kwargs={"k": 10},
    search_type="similarity"
)

retr_ent = entities.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 4}
)

retr_fea = features.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 4}
)

In [13]:
datas = {}
datas[0] = {
    'question': mTests['AVTOKFLEKP-T692']
}
print(f'\n\nquestion: {datas[0]['question']}')



question: Key;Name;Test Step;Expected Result: AVTOKFLEKP-T692;AVTOKFLEKP-608_ИФТ_03(УСПЕХ) Сервис Efsgetcontractinfo (инфо-запрос). В АС ЕКП КД НАШЛИ.;Проверить
None: что запрос Efsgetcontractinfo: поступил в АС ЕКП,КД найден и в АС ?ЕФС? отправлен успешный отклик с status/statusCode = 0 и с блоком loan,который содержит данные по найденному КД.;Запрос Efsgetcontractinfo: поступил в АС ЕКП,КД найден и в АС ?ЕФС? отправлен успешный отклик с status/statusCode = 0 и с блоком loan,который содержит данные по найденному КД.


In [14]:
datas[0]['key_words'] = llm_get_objects(datas[0]['question'], llm)
print(f'{datas[0]['key_words']}')

AVTOKFLEKP-T692  
AVTOKFLEKP-608_ИФТ_03(УСПЕХ)  
Efsgetcontractinfo  
АС ЕКП  
КД  
loan  
statusCode


In [15]:
retr_tests_data = retr_tests.invoke(datas[0]['question'])
rel_test_keys = [x.metadata['test_key'] for x in retr_tests_data]
datas[0]['relevant_tests'] = format_docs(retr_tests_data)
print(f'{datas[0]['relevant_tests']}')

Key;Name;Test Step;Expected Result: AVTOKFLEKP-T692;AVTOKFLEKP-608_ИФТ_03(УСПЕХ) Сервис Efsgetcontractinfo (инфо-запрос). В АС ЕКП КД НАШЛИ.;Проверить
None: что запрос Efsgetcontractinfo: поступил в АС ЕКП,КД найден и в АС ?ЕФС? отправлен успешный отклик с status/statusCode = 0 и с блоком loan,который содержит данные по найденному КД.;Запрос Efsgetcontractinfo: поступил в АС ЕКП,КД найден и в АС ?ЕФС? отправлен успешный отклик с status/statusCode = 0 и с блоком loan,который содержит данные по найденному КД.

Key;Name;Test Step;Expected Result: AVTOKFLEKP-T693;AVTOKFLEKP-608_ИФТ_04(ОШИБКА) Сервис Efsgetcontractinfo (инфо-запрос). В АС ЕКП КД НЕ нашли.;Проверить
None: что запрос Efsgetcontractinfo: поступил в АС ЕКП,КД НЕ найден и в АС ?ЕФС? отправлен НЕуспешный отклик с status/statusCode = 4,status/errorCode= 1,status/errorMessage= Кредит не найден } и БЕЗ блока loan.;Запрос Efsgetcontractinfo: поступил в АС ЕКП,КД НЕ найден и в АС ?ЕФС? отправлен НЕуспешный отклик с status/statusCode =

In [16]:
high_level_steps_prompt = ChatPromptTemplate.from_messages([
        ("system", '''Ты — системный аналитик с опытом написания подробных процессов.
            Ты получишь [ЗАПРОС] и релевантные [ТЕСТЫ]. Проанализируй их, дополни [ЗАПРОС] недостающими данными из [ТЕСТОВ].
            Ты можешь искать сущности в базе, менять, обновлять.
            Ты можешь делать запросы через сервис, работать с их запросами и ответами.
            Сгенерируй шаги для выполнения [ЗАПРОСА].
        
            Размышляй шаг за шагом!'''),
        ("human", '''[ТЕСТЫ]: {relevant_tests}. 
        Ключевые слова: {key_words}. 
        [ЗАПРОС]: {question}'''),
    ])

In [17]:
with get_openai_callback() as cb:

    chain = RunnablePassthrough.assign(
        high_steps=(high_level_steps_prompt | llm | StrOutputParser())
    )

    datas[0] = chain.invoke(datas[0])

    print(datas[0]['high_steps'])

    print(f"\n\nTotal Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")

### Шаги для выполнения запроса:

1. **Идентификация тестового кейса**:
   - Найти тестовый кейс с ключом `AVTOKFLEKP-T692` и названием `AVTOKFLEKP-608_ИФТ_03(УСПЕХ)` в системе тестирования.
   
2. **Подготовка данных**:
   - Убедиться, что в тестовом окружении есть контракт (КД), который можно найти по идентификатору (`ID_KD`), используемому в тесте.
   - Подготовить запрос сервиса `Efsgetcontractinfo`, включив в него правильный идентификатор контракта (`ID_KD`).

3. **Отправка запроса**:
   - Отправить подготовленный запрос сервиса `Efsgetcontractinfo` в автоматизированную систему ЕКП (АС ЕКП).

4. **Мониторинг обработки запроса**:
   - Отследить поступление запроса в АС ЕКП и убедиться, что система обработала его успешно.

5. **Получение отклика от системы**:
   - Получить отклик от АС ЕКП и проверить, что он был отправлен обратно в АС ЕФС.

6. **Анализ результата**:
   - Проверить, что отклик содержит статус `statusCode = 0`, указывающий на успешное выполнение операции.
   - Провер

In [18]:
mid_level_steps_prompt = ChatPromptTemplate.from_messages([
        ("system", '''Ты — системный аналитик с опытом написания подробных процессов.
            Ты получишь список [ШАГОВ] для выполнения запроса и релевантные [ДАННЫЕ]. Проанализируй их, дополни [ШАГИ] недостающими данными из [ДАННЫЕ].
            Сгенерируй шаги для выполнения запроса.
        
            Размышляй шаг за шагом!'''),
        ("human", '''[ДАННЫЕ]: {entities}.
        Ключевые слова: {key_words}. 
        [ШАГИ]: {high_steps}'''),
    ])

In [19]:
test_dtos_lines = [x.page_content for x in get_splitted_doc(file_path='../monoscript/test-dto.txt', chuck_size=0, chuck_overlap=0)]

Created a chunk of size 45, which is longer than the specified 0
Created a chunk of size 45, which is longer than the specified 0
Created a chunk of size 45, which is longer than the specified 0
Created a chunk of size 45, which is longer than the specified 0
Created a chunk of size 38, which is longer than the specified 0
Created a chunk of size 45, which is longer than the specified 0
Created a chunk of size 45, which is longer than the specified 0
Created a chunk of size 52, which is longer than the specified 0
Created a chunk of size 104, which is longer than the specified 0
Created a chunk of size 45, which is longer than the specified 0
Created a chunk of size 45, which is longer than the specified 0
Created a chunk of size 44, which is longer than the specified 0
Created a chunk of size 37, which is longer than the specified 0
Created a chunk of size 37, which is longer than the specified 0
Created a chunk of size 37, which is longer than the specified 0
Created a chunk of size 

In [20]:
pat3 = r'(\w+-T\d+)=(.+)'
test_dtos = {}
for test_dto_line in test_dtos_lines:
    matcher = re.match(pat3, test_dto_line)
    if matcher:
        test_dtos[matcher.group(1)] = [x for x in matcher.group(2).split(",")] 

In [21]:
from itertools import chain
unique_elements = set(chain.from_iterable([test_dtos[x1] for x1 in rel_test_keys]))
ent_docs = [mEntities[ent] for ent in unique_elements]
ent_docs

[Document(metadata={'source': '../monoscript/src/main/java/ru/sber/cb/ekp/avtokflekp/subsystem/loans/service/os/SrvGetContractInfo/dto/response/ContractRs.java', 'class_key': 'ContractRs'}, page_content='package ru.sber.cb.ekp.avtokflekp.subsystem.loans.service.os.SrvGetContractInfo.dto.response;\n\nimport com.fasterxml.jackson.annotation.JsonProperty;\nimport lombok.*;\nimport ru.sber.cb.ekp.annotation.StepDto;\n\n@Data\n@NoArgsConstructor\npublic class ContractRs {\n    @JsonProperty("loan")\n    private Loan loan;\n    @JsonProperty("status")\n    private Status status;\n}\n'),
 Document(metadata={'source': '../monoscript/src/main/java/ru/sber/cb/ekp/avtokflekp/subsystem/loans/service/os/SrvGetContractInfo/dto/request/ContractRq.java', 'class_key': 'ContractRq'}, page_content='package ru.sber.cb.ekp.avtokflekp.subsystem.loans.service.os.SrvGetContractInfo.dto.request;\n\nimport lombok.Data;\nimport lombok.NoArgsConstructor;\n\n@Data\n@NoArgsConstructor\npublic class ContractRq {\n  

In [22]:
datas[1] = {
    'entities': format_docs(ent_docs),
    'key_words': datas[0]['key_words'],
    'high_steps': datas[0]['high_steps'],
}

In [23]:
with get_openai_callback() as cb:

    chain = RunnablePassthrough.assign(
        mid_steps=(mid_level_steps_prompt | llm | StrOutputParser())
    )

    datas[1] = chain.invoke(datas[1])

    print(datas[1]['mid_steps'])

    print(f"\n\nTotal Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")

### Дополненные шаги для выполнения запроса:

1. **Идентификация тестового кейса**:
   - Найти тестовый кейс с ключом `AVTOKFLEKP-T692` и названием `AVTOKFLEKP-608_ИФТ_03(УСПЕХ)` в системе тестирования.
   
2. **Подготовка данных**:
   - Убедиться, что в тестовом окружении есть контракт (КД), который можно найти по идентификатору (`productId`), используемому в тесте.
   - Создать объект класса `ContractRq` и установить значение поля `productId` равным идентификатору существующего контракта.
   - Установить поле `system` объекта `ContractRq` равным значению, соответствующему системе, из которой отправляется запрос.

3. **Отправка запроса**:
   - Использовать сервис `SrvGetContractInfo` для отправки запроса, передавая созданный объект `ContractRq`.
   - Сервис вернет объект типа `ContractRs`.

4. **Мониторинг обработки запроса**:
   - Отследить поступление запроса в АС ЕКП и убедиться, что система обработала его успешно.

5. **Получение отклика от системы**:
   - Извлечь объект `Contract

In [34]:
translate_level_steps_prompt = ChatPromptTemplate.from_messages([
        ("system", '''Ты — опытный разработчик автотестов Cucumber-gherkin.
            Ты получишь [ПРАВИЛА] работы в проекте, [СЦЕНАРИЙ] для автоматизации и [ПРИМЕРЫ] автоматизации для аналогичных процессов.
            1. Проанализируй примеры.
            2. Сгенерируй feature файл для выполнения [СЦЕНАРИЯ] на языке gherkin. 
            3. Убедись, что тест не содержит ничего кроме рабочего кода и комментариев, связанных со [СЦЕНАРИЕМ].
            
            Используй только существующие [ШАГИ] из [ПРИМЕРОВ]. 
            Сопоставляй [ШАГИ] из [СЦЕНАРИЯ] пошагово. Если похожего шага не существует - напиши вместо него комментарий с возможной трактовкой.
            '''),
        ("human", '''[ПРИМЕРЫ]: {fewshot_examples}
         [ШАГИ]: {steps}
         [ПРАВИЛА]: {rules}
        [СЦЕНАРИЙ]: {scenario}'''),
    ])

In [35]:
datas[2] = {
    'fewshot_examples': "\n".join([f"Q: {x}\nA: {mFeature.get(x)}" for x in rel_test_keys]),
    'steps': format_docs(retr_steps.invoke(",".join(unique_elements))),
    'rules': get_doc('prompts/rules.txt'),
    'scenario': datas[1]['mid_steps']
}

In [36]:
print(f"{datas[2]['steps']}")

@And(Ищу PrCred по условию Casco1)
@And(Ищу PrCred по условию Casco9)
@And(Ищу PrCred по условию Id)
@And(Ищу PrCred по условию KindCredit_Code1)
@And(Ищу PrCred по условию StatusAndKindAndNoZalog)
@And(Ищу PrCred по условию StatusAndKind)
@And(Ищу PrCred по условию StatusAndKindWithZalog)
@And(Ищу PrCred по условию StatusAndKindNoEcm)
@And(Ищу PrCred по условию StatusAndKindWithRates)
@And(Ищу PrCred по условию StatusAndKindNoZalogEstate)
@And(Ищу PrCred по условию StatusAndKindWithProperty)
@And(Ищу PrCred по условию StatusAndKindWithoutRates)
@And(Ищу PrCred по условию StatusAndKindAndBeforeDate)
@And(Ищу PrCred по условию StatusAndNotLikeKindNameAndBeforeDateByDateBeginDesc)
@And(Ищу PrCred по условию StatusAndKindNameAndBeforeDate)


@And(Ищу ErmbSms по условию PatternAndPrCred)
@And(Удаляю ErmbSms по условию PatternAndPrCred)


@And(Ищу SbeInfSmsPat по условию Code)


@And(Ищу Bgp по условию ShortName)

@And(Сохраняю ContractRs.loan в {context_key})
@Then(Проверяю что ContractRs.

In [37]:
llm0 = get_llm(temperature=0.0)

In [38]:
with get_openai_callback() as cb:

    chain = RunnablePassthrough.assign(
        result_test=(translate_level_steps_prompt | llm0 | StrOutputParser())
    )

    datas[2] = chain.invoke(datas[2])

    print(datas[2]['result_test'])

    print(f"\n\nTotal Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")

@rt @ci
@AVTOKFLEKP-T692
@tmsLink=AVTOKFLEKP-T692
@issue=AVTOKFLEKP-T692
@link=https://jira.delta.sbrf.ru/secure/Tests.jspa#/testCase/AVTOKFLEKP-T692
Feature: T692
  Scenario:
    Given Ищу PrCred по условию StatusAndKind
      | status  | "WORK"    |
      | kind    | "2.107.1" |

    And Сохраняю PrCred.numDog в <numDog>

    And Генерирую запрос ContractRq для сервиса SrvGetContractInfo

    And Записываю <numDog> в ContractRq.productId

    And Отправляю POST запрос ContractRq в сервис SrvGetContractInfo

    Then Проверяю что ContractRs.status.statusCode == 0

    Then Проверяю что ContractRs.loan != null

    Then Проверяю что ContractRs.loan.agreementNumber == <numDog>


Total Tokens: 1069
Prompt Tokens: 836
Completion Tokens: 233
